# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Consumer** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

# Creacion de la conexión con el cluster de spark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://80d04dce9402:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

### Creación del Kafka Stream

In [ ]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, FloatType

schema = StructType() \
    .add("price", FloatType()) \
    .add("s", StringType()) \
    .add("event_time", StringType()) \
    .add("v", FloatType())

kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "8e35f34f36db:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

### Transform binary data into string

In [ ]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

parsed_df = kafka_lines.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.*")

In [ ]:
# Save as Parquet
query = parsed_df.writeStream \
    .format("parquet") \
    .option("path", "/tmp/financial-data/") \
    .option("checkpointLocation", "/tmp/checkpoints/") \
    .outputMode("append") \
    .start()

query.awaitTermination()

In [ ]:
spark.read.parquet("/tmp/financial-data/").show()

In [ ]:
query.stop()

In [ ]:
sc.stop()